In [1]:
import tensorflow as tf
# from tensorflow.keras import layers
from tensorflow import keras
from keras import layers
from sklearn import  pipeline, preprocessing
from torchvision.datasets import EMNIST

from torch.utils.data import DataLoader

2024-01-31 21:36:17.359678: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 21:36:17.391299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 21:36:17.391328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 21:36:17.392050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 21:36:17.398677: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-01-31 21:36:20.655950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 21:36:20.657075: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 21:36:20.657199: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:


# Load the datasets
train_data = EMNIST(root='emnist_data/', split='byclass', train=True, download=True)
test_data = EMNIST(root='emnist_data/', split='byclass', train=False, download=True)

# train_data_loader = DataLoader(train_data, batch_size=64, shuffle=True)
# test_data_loader = DataLoader(test_data, batch_size=64, shuffle=True)

# train_images = tf.data.Dataset.from_generator(lambda: train_data_loader, output_signature=tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))
# test_images = tf.data.Dataset.from_generator(lambda: test_data_loader, output_signature=tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32))
#unpack the data into images and labels
train_images, train_labels = train_data.data, train_data.targets
test_images, test_labels = test_data.data, test_data.targets

#convert the data to numpy arrays
train_images = train_images.numpy()
train_labels = train_labels.numpy()
test_images = test_images.numpy()
test_labels = test_labels.numpy()


In [4]:
print(len(train_data))
print(len(test_data))

697932
116323


In [5]:
data_preprocessing_pipeline = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda X: tf.convert_to_tensor(X))
)

train_images = data_preprocessing_pipeline.fit_transform(train_images)
train_labels = data_preprocessing_pipeline.fit_transform(train_labels)

test_images = data_preprocessing_pipeline.fit_transform(test_images)
test_labels = data_preprocessing_pipeline.fit_transform(test_labels)

2024-01-31 21:36:21.301487: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 21:36:21.301701: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 21:36:21.301778: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
testing_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [7]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (5,5), activation='tanh', padding = 'same', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(48, (5,5), activation='tanh', padding = 'same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (5,5), activation='tanh', padding = 'same'),
    layers.Flatten(),
    layers.Dense(256, activation='tanh'),
    layers.Dense(128, activation='tanh'),
    layers.Dense(62, activation='softmax')
])

In [8]:

model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [9]:
# the training loop
epochs = 10
batch_size = 256

model.fit(training_dataset.shuffle(10000).batch(batch_size), epochs=epochs, batch_size=batch_size, validation_data=testing_dataset.batch(batch_size))

Epoch 1/10


2024-01-31 21:36:22.898388: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-01-31 21:36:23.622693: I external/local_xla/xla/service/service.cc:168] XLA service 0x7feb5c0288d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 21:36:23.622720: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-01-31 21:36:23.627777: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1706726183.694721   10082 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2727/2727 [==============================] - 36s 12ms/step - loss: 0.5005 - accuracy: 0.8314 - val_loss: 0.4308 - val_accuracy: 0.8451
Epoch 2/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.4208 - accuracy: 0.8493 - val_loss: 0.4208 - val_accuracy: 0.8492
Epoch 3/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.4121 - accuracy: 0.8514 - val_loss: 0.4126 - val_accuracy: 0.8519
Epoch 4/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.4029 - accuracy: 0.8540 - val_loss: 0.4045 - val_accuracy: 0.8538
Epoch 5/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.4025 - accuracy: 0.8541 - val_loss: 0.4099 - val_accuracy: 0.8524
Epoch 6/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.3975 - accuracy: 0.8559 - val_loss: 0.4048 - val_accuracy: 0.8533
Epoch 7/10
2727/2727 [==============================] - 31s 11ms/step - loss: 0.3933 - accuracy: 0.8566 - val_loss: 0.4043 - val_accura

In [10]:
#evaluating the model with unseen data
model.evaluate(testing_dataset.batch(1))


116323/116323 [==============================] - 166s 1ms/step - loss: 0.3987 - accuracy: 0.8552


[0.39869552850723267, 0.8552048802375793]